In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
train_file = "../temp/data/train.csv"
eval_file = "../temp/data/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

print(train_df.head())
print(eval_df.head())

In [ ]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

print(train_df.head())
print(y_train.head())
print(eval_df.head())
print(y_eval.head())

In [ ]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

feature_columns = []
for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    print(vocab)
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(categorical_column, vocab)))

for numeric_column in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(numeric_column, dtype=tf.float32))

In [ ]:
def make_dataset(data_df, label_df, epochs = 100, shuffle = True, batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
        datase = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [ ]:
# not working, don't know why
"""
output_dir = '../temp/baseline_model'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

baseline_estimator = tf.estimator.BaselineClassifier(model_dir=output_dir, n_classes=2)
baseline_estimator.train(input_fn=lambda: make_dataset(train_df, y_train, epochs=100))
"""

In [ ]:
# not working, don't know why
"""
baseline_estimator.evaluate(input_fn=lambda: make_dataset(eval_df, y_eval, epochs=1, shuffle=False, batch_size=20))
"""

In [ ]:
linear_output_dir = "../temp/linear_model"
if not os.path.exists(linear_output_dir):
    os.mkdir(linear_output_dir)

linear_estimator = tf.estimator.LinearClassifier(model_dir=linear_output_dir, n_classes=2, feature_columns=feature_columns)
linear_estimator.train(input_fn=lambda: make_dataset(train_df, y_train, epochs=100))

In [ ]:
linear_estimator.evaluate(input_fn=lambda: make_dataset(eval_df, y_eval, epochs=1, shuffle=False))

In [ ]:
dnn_output_dir = "../temp/dnn_model"

if not os.path.exists(dnn_output_dir):
    os.mkdir(dnn_output_dir)

dnn_estimator = tf.estimator.DNNClassifier(model_dir=dnn_output_dir,
                                          n_classes=2,
                                          feature_columns=feature_columns,
                                          hidden_units=[128, 128],
                                          activation_fn=tf.nn.relu,
                                          optimizer="Adam")
dnn_estimator.train(input_fn=lambda: make_dataset(train_df, y_train, epochs=100))

In [ ]:
dnn_estimator.evaluate(input_fn=lambda: make_dataset(eval_df, y_eval, epochs=1, shuffle=False))